In [1]:
import pandas as pd

In [2]:
state_list = ['CA', 'TX', 'FL', 'NH', 'PA']

## Link FIPS & HOSP data

#### hosp data

In [42]:
hosp_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Outcome/hosp10-reports/HOSPITAL10_PROVIDER_ID_INFO.CSV')
hosp_df = hosp_df[['PROVIDER_NUMBER', 'HOSP10_Name',
                   'State_id', 'City', 'County','zip_code_clean']]
hosp_df = hosp_df.sort_values(by=['PROVIDER_NUMBER'])
hosp_df = hosp_df[hosp_df['State_id'].isin(state_list)]
hosp_df = hosp_df.reset_index(drop=True)

# Clean county names for FIPS link
hosp_df['County'] = hosp_df['County'].replace(to_replace={"( COUNTY)":"", "( County)":"", "( county)":"",
                                                                              "( BOROUH)":"", "( BOROUGH)":"","( Borough)":"", "( borough)":"",
                                                                              "( PARISH)":"", "( Parish)":"", "( parish)":"",
                                                                              "( MUNICIPALITY)":"", "( Municipality)":"", "( municipality)":"",
                                                                              "( CITY)":"", "( City)":"", "( city)":"",
                                                                              "( PENINSULA)":"", "( Peninsula)":"", "( peninsula)":"",
                                                                              "( THE DISTRICT)":"dc", "(ST. )":"st. ", "(SAINT )":"st. "}, regex=True)

# 고치기
hosp_df['county_clean'] = [i.lower() for i in hosp_df['County']]
hosp_df['county_clean'] = [i.replace(" ","") for i in hosp_df['county_clean']]
hosp_df['county_clean'] = hosp_df['county_clean'].replace(to_replace=
                                                                    ['[\\u0001-\\u002F]','[\\u003A-\\u0040]',
                                                                     '[\\u005B-\\u0060]','[\\u007B-\\u9999]',
                                                                     '[＜＞]'], value='', regex=True)

# Index
hosp_df['state_county'] = [str(hosp_df['State_id'][i])+"_"+str(hosp_df['county_clean'][i]) for i in range(len(hosp_df))]

#### Fips data

In [44]:
fips_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/FIPS/fips_df2.csv')

# clean
new_list = []
for i in range(len(fips_df)):
    if len(str(fips_df['FIPS'][i]))==4:
        new_list.append(str(0)+str(fips_df['FIPS'][i]))
    elif len(str(fips_df['FIPS'][i]))==5:
        new_list.append(str(fips_df['FIPS'][i]))
    else:
        print(i)
fips_df['FIPS'] = new_list

# Merge
hosp_df = pd.merge(hosp_df, fips_df, on=['State_id', 'state_county'])
# clean
hosp_df = hosp_df.drop(columns = ['county_clean_x'])
hosp_df = hosp_df.rename(columns = {'county_clean_y':'county_clean'})

----

### Cost Report

In [46]:
cost_2018_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Outcome/Medicare_Inpatient/Medicare_Provider_Charge_Inpatient_DRGALL_FY2018.csv')
cost_2017_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Outcome/Medicare_Inpatient/Medicare_Provider_Charge_Inpatient_DRGALL_FY2017.csv')

# FY
cost_2018_df['FY'] = 2018
cost_2017_df['FY'] = 2017
# DRG
cost_2018_df['DRG'] = [i[:3] for i in cost_2018_df['DRG_DESC']]
cost_2017_df['DRG'] = [i[:3] for i in cost_2017_df['DRG Definition']]
# Focus states
cost_2018_df = cost_2018_df[cost_2018_df['STATE_DESC'].isin(state_list)]
cost_2017_df = cost_2017_df[cost_2017_df['Provider State'].isin(state_list)]

hosp_df = hosp_df.rename(columns = {"PROVIDER_NUMBER":"PROVIDER_ID"})
cost_2018_df = pd.merge(cost_2018_df, hosp_df, on=['PROVIDER_ID'])

hosp_df = hosp_df.rename(columns = {'PROVIDER_ID':'Provider Id'})
cost_2017_df = pd.merge(cost_2017_df, hosp_df, on=['Provider Id'])

In [51]:
# make columns
## 큰 순서대로 
# Average Covered Charges: Hospital's average charge for services covered by medicare for all discharges in the MS-DRG.
# Average Total Payments: What Medicare actually pays to the provider as well as co-payment and deductible amounts that the beneficiary is responsbile for and payments by third parties for coordination of benefits.
# Average Medicare Payments: The average amount that Medicare pays to the hospital for Medicare's share of the MS-DRG.

cost_2018_df['TOTAL_COVERED_CHARGES'] = cost_2018_df['DISCHARGE_COUNT_SUM']*cost_2018_df['MEAN_COVERED_CHARGES'] 
cost_2018_df['TOTAL_PAYMENTS'] = cost_2018_df['DISCHARGE_COUNT_SUM']*cost_2018_df['MEAN_MEDICARE_PAYMENTS'] 
cost_2018_df['TOTAL_MEDICARE_PAYMENTS'] = cost_2018_df['DISCHARGE_COUNT_SUM']*cost_2018_df['MEAN_MEDICARE_REIMBURSEMENT'] 

cost_2017_df['TOTAL_COVERED_CHARGES'] = cost_2017_df['Total Discharges'] * cost_2017_df['Average Covered Charges']
cost_2017_df['TOTAL_PAYMENTS'] = cost_2017_df['Total Discharges'] * cost_2017_df['Average Total Payments']
cost_2017_df['TOTAL_MEDICARE_PAYMENTS'] = cost_2017_df['Total Discharges'] * cost_2017_df['Average Medicare Payments']

cost_2018_df = cost_2018_df[['FIPS', 'State_id','County', 'DRG_DESC', 
                             'DISCHARGE_COUNT_SUM', 'TOTAL_COVERED_CHARGES', 'TOTAL_PAYMENTS','TOTAL_MEDICARE_PAYMENTS',
                             'State_FIPS','County_FIPS','county_clean',
                             'PROVIDER_ID', 'FACILITY_NAME', 'FACILITY_CITY', 'FY', 'DRG','zip_code_clean']]
cost_2017_df = cost_2017_df[['FIPS', 'State_id', 'County', 'DRG Definition', 
                             'Total Discharges', 'TOTAL_COVERED_CHARGES', 'TOTAL_PAYMENTS','TOTAL_MEDICARE_PAYMENTS',
                             'State_FIPS','County_FIPS','county_clean',
                             'Provider Id', 'Provider Name','Provider City', 'FY', 'DRG','zip_code_clean']]

##### MS-DRG

In [76]:
# Crawl from wiki (https://en.wikipedia.org/wiki/Major_Diagnostic_Category)
import pandas as pd
import requests, re, os
import urllib
from bs4 import BeautifulSoup as bs

html = requests.get('https://en.wikipedia.org/wiki/Major_Diagnostic_Category')
soup = bs(html.text, 'html.parser')
html.close()

table = soup.find('table', {'class':'wikitable'})
trs = table.find_all('tr')

new_dict = {}
for i in range(len(trs)):
    if i>=1:
        new_dict[trs[i].find_all('td')[-1].get_text(strip=True)] = trs[i].find_all('td')[0].get_text(strip=True)
    else:
        print(i)

# to df
drg_df = pd.DataFrame.from_dict(data=new_dict, orient='index', columns=['description'])
drg_df.reset_index(inplace=True)
drg_df.columns = ['msdrg','description']

## tuning
drg_df['msdrg_start'] = [i[:3] for i in drg_df['msdrg']]
drg_df['msdrg_end'] = [i[-3:] for i in drg_df['msdrg']]

## dtypes
drg_df['msdrg_start'] = drg_df['msdrg_start'].astype(int)
drg_df['msdrg_end'] = drg_df['msdrg_end'].astype(int)

In [124]:
## 2018
cost_2018_df['DRG'] = cost_2018_df['DRG'].astype(int)

# meta
cost_2018_df_meta = cost_2018_df[['DRG']].drop_duplicates()
cost_2018_df_meta = cost_2018_df_meta.sort_values(by=['DRG'])
cost_2018_df_meta = cost_2018_df_meta.reset_index(drop=True)

# classify
temp_list = []
new_dict = {}
for i in range(len(cost_2018_df_meta)):
    for j in range(len(drg_df)):
        if (cost_2018_df_meta['DRG'][i]>=drg_df['msdrg_start'][j])&(cost_2018_df_meta['DRG'][i]<=drg_df['msdrg_end'][j]):
            new_dict[cost_2018_df_meta['DRG'][i]] = drg_df['description'][j]
# to df
temp_df = pd.DataFrame.from_dict(data=new_dict,orient='index').reset_index(level=[0])
temp_df.columns = ['DRG','description']

## tuning (missing)
[i for i in cost_2018_df_meta.DRG.values.tolist() if i not in temp_df.DRG.values.tolist()]  # 570, 571
new_dict[570] = 'Diseases and Disorders of the Skin, Subcutaneous Tissue And Breast'
new_dict[571] = 'Diseases and Disorders of the Skin, Subcutaneous Tissue And Breast'
temp_df = pd.DataFrame.from_dict(data=new_dict,orient='index').reset_index(level=[0])
temp_df.columns = ['DRG','description'] 
       
# Merge
cost_2018_df = pd.merge(cost_2018_df, temp_df, on=['DRG'])



## 2017
cost_2017_df['DRG'] = cost_2017_df['DRG'].astype(int)

# meta
cost_2017_df_meta = cost_2017_df[['DRG']].drop_duplicates()
cost_2017_df_meta = cost_2017_df_meta.sort_values(by=['DRG'])
cost_2017_df_meta = cost_2017_df_meta.reset_index(drop=True)

# classify
temp_list = []
new_dict = {}
for i in range(len(cost_2017_df_meta)):
    for j in range(len(drg_df)):
        if (cost_2017_df_meta['DRG'][i]>=drg_df['msdrg_start'][j])&(cost_2017_df_meta['DRG'][i]<=drg_df['msdrg_end'][j]):
            new_dict[cost_2017_df_meta['DRG'][i]] = drg_df['description'][j]
# to df
temp_df = pd.DataFrame.from_dict(data=new_dict,orient='index').reset_index(level=[0])
temp_df.columns = ['DRG','description']

## tuning (missing)
[i for i in cost_2017_df_meta.DRG.values.tolist() if i not in temp_df.DRG.values.tolist()]  # 570, 571
new_dict[570] = 'Diseases and Disorders of the Skin, Subcutaneous Tissue And Breast'
new_dict[571] = 'Diseases and Disorders of the Skin, Subcutaneous Tissue And Breast'
temp_df = pd.DataFrame.from_dict(data=new_dict,orient='index').reset_index(level=[0])
temp_df.columns = ['DRG','description'] 

# Merge
cost_2017_df = pd.merge(cost_2017_df, temp_df, on=['DRG'])

#### Groupby by ['DRG', 'FIPS']

In [225]:
## 2018
cost_2018_df_gb = cost_2018_df.groupby(['FIPS','description']).sum()[['DISCHARGE_COUNT_SUM', 'TOTAL_COVERED_CHARGES', 
                                                                      'TOTAL_PAYMENTS', 'TOTAL_MEDICARE_PAYMENTS']].reset_index(level=[0,1])
cost_2018_df_meta = cost_2018_df[['FIPS', 'description', 'State_id','FY']].drop_duplicates()
cost_2018_df_fips = pd.merge(cost_2018_df_gb, cost_2018_df_meta, on=['FIPS','description'])

## 2017
cost_2017_df_gb = cost_2017_df.groupby(['FIPS','description']).sum()[['Total Discharges', 'TOTAL_COVERED_CHARGES', 
                                                                      'TOTAL_PAYMENTS', 'TOTAL_MEDICARE_PAYMENTS']].reset_index(level=[0,1])
cost_2017_df_meta = cost_2017_df[['FIPS', 'description', 'State_id','FY']].drop_duplicates()
cost_2017_df_fips = pd.merge(cost_2017_df_gb, cost_2017_df_meta, on=['FIPS','description'])

### IV

In [236]:
iv_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/IV/Market_Saturation_And_Utilization_Dataset_2021-04-16.csv')
iv_df = iv_df[iv_df['Type_of_Service']=='Telemedicine']
iv_df = iv_df[iv_df['Aggregation_Level']=='COUNTY'] # County

# Columns of interest
iv_df = iv_df[['Reference_Period', 'Type_of_Service', 'Aggregation_Level', 'State',
               'County', 'State_FIPS', 'County_FIPS',
               'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers',
               'Number_of_Users','Total_Payment']]

# Focus state
iv_df = iv_df[iv_df['State'].isin(state_list)]
iv_df = iv_df.reset_index(drop=True) 

# Clean FIPS
## State
new_list = []
for i in range(len(iv_df)):
    if len(str(int(iv_df['State_FIPS'][i])))==1:
        new_list.append(str('0')+str(int(iv_df['State_FIPS'][i])))
    elif len(str(int(iv_df['State_FIPS'][i])))==2:
        new_list.append(str(int(iv_df['State_FIPS'][i])))
    else:
        print(i)
iv_df['State_FIPS'] = new_list

## County
new_list = []
for i in range(len(iv_df)):
    if len(str(int(iv_df['County_FIPS'][i])))==1:
        new_list.append(str('00')+str(int(iv_df['County_FIPS'][i])))
    elif len(str(int(iv_df['County_FIPS'][i])))==2:
        new_list.append(str('0')+str(int(iv_df['County_FIPS'][i])))
    elif len(str(int(iv_df['County_FIPS'][i])))==3:
        new_list.append(str(int(iv_df['County_FIPS'][i])))
    else:
        print(i)
iv_df['County_FIPS'] = new_list

## Both
iv_df['FIPS'] = [str(iv_df['State_FIPS'][i])+str(iv_df['County_FIPS'][i]) for i in range(len(iv_df))]

## Reference dates
iv_2018_df = iv_df[iv_df['Reference_Period']=='2018-01-01 to 2018-12-31']
iv_2017_df = iv_df[iv_df['Reference_Period']=='2017-01-01 to 2017-12-31']

iv_dv_2018_df = pd.merge(cost_2018_df_fips, iv_2018_df, on=['FIPS'])
iv_dv_2017_df = pd.merge(cost_2017_df_fips, iv_2017_df, on=['FIPS'])

----

In [284]:
iv_dv_2018_df.description.value_counts()

Diseases and Disorders of the Respiratory System                                            222
Diseases and Disorders of the Kidney And Urinary Tract                                      216
Infectious and Parasitic DDs (Systemic or unspecified sites)                                215
Diseases and Disorders of the Circulatory System                                            211
Diseases and Disorders of the Digestive System                                              196
Diseases and Disorders of the Musculoskeletal System And Connective Tissue                  194
Diseases and Disorders of the Skin, Subcutaneous Tissue And Breast                          192
Diseases and Disorders of the Endocrine, Nutritional And Metabolic System                   190
Diseases and Disorders of the Nervous System                                                166
Diseases and Disorders of the Blood and Blood Forming Organs and Immunological Disorders    148
Diseases and Disorders of the Hepatobili

In [289]:
resp_2018_df = iv_dv_2018_df[iv_dv_2018_df['description']=='Diseases and Disorders of the Respiratory System']
resp_2017_df = iv_dv_2017_df[iv_dv_2017_df['description']=='Diseases and Disorders of the Respiratory System']

kidney_2018_df = iv_dv_2018_df[iv_dv_2018_df['description']=='Diseases and Disorders of the Kidney And Urinary Tract']
kidney_2017_df = iv_dv_2017_df[iv_dv_2017_df['description']=='Diseases and Disorders of the Kidney And Urinary Tract']

infect_2018_df = iv_dv_2018_df[iv_dv_2018_df['description']=='Infectious and Parasitic DDs (Systemic or unspecified sites)']
infect_2017_df = iv_dv_2017_df[iv_dv_2017_df['description']=='Infectious and Parasitic DDs (Systemic or unspecified sites)']

circ_2018_df = iv_dv_2018_df[iv_dv_2018_df['description']=='Diseases and Disorders of the Circulatory System']
circ_2017_df = iv_dv_2017_df[iv_dv_2017_df['description']=='Diseases and Disorders of the Circulatory System']

In [301]:
resp_2018_df = resp_2018_df[['FIPS',
                             'DISCHARGE_COUNT_SUM', 'TOTAL_COVERED_CHARGES','TOTAL_PAYMENTS', 'TOTAL_MEDICARE_PAYMENTS', 
                             'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers','Number_of_Users', 'Total_Payment']]
resp_2017_df = resp_2017_df[['FIPS', 
                             'Total Discharges', 'TOTAL_COVERED_CHARGES','TOTAL_PAYMENTS', 'TOTAL_MEDICARE_PAYMENTS', 
                             'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers', 'Number_of_Users', 'Total_Payment']]

kidney_2018_df = kidney_2018_df[['FIPS',
                             'DISCHARGE_COUNT_SUM', 'TOTAL_COVERED_CHARGES','TOTAL_PAYMENTS', 'TOTAL_MEDICARE_PAYMENTS', 
                             'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers','Number_of_Users', 'Total_Payment']]
kidney_2017_df = kidney_2017_df[['FIPS', 
                             'Total Discharges', 'TOTAL_COVERED_CHARGES','TOTAL_PAYMENTS', 'TOTAL_MEDICARE_PAYMENTS', 
                             'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers', 'Number_of_Users', 'Total_Payment']]

infect_2018_df = infect_2018_df[['FIPS',
                             'DISCHARGE_COUNT_SUM', 'TOTAL_COVERED_CHARGES','TOTAL_PAYMENTS', 'TOTAL_MEDICARE_PAYMENTS', 
                             'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers','Number_of_Users', 'Total_Payment']]
infect_2017_df = infect_2017_df[['FIPS', 
                             'Total Discharges', 'TOTAL_COVERED_CHARGES','TOTAL_PAYMENTS', 'TOTAL_MEDICARE_PAYMENTS', 
                             'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers', 'Number_of_Users', 'Total_Payment']]

circ_2018_df = circ_2018_df[['FIPS',
                             'DISCHARGE_COUNT_SUM', 'TOTAL_COVERED_CHARGES','TOTAL_PAYMENTS', 'TOTAL_MEDICARE_PAYMENTS', 
                             'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers','Number_of_Users', 'Total_Payment']]
circ_2017_df = circ_2017_df[['FIPS', 
                             'Total Discharges', 'TOTAL_COVERED_CHARGES','TOTAL_PAYMENTS', 'TOTAL_MEDICARE_PAYMENTS', 
                             'Number_of_Fee-for-Service_Beneficiaries', 'Number_of_Providers', 'Number_of_Users', 'Total_Payment']]

In [313]:
# rename columns for merge
cols = resp_2018_df.columns
resp_2018_df.columns = [str(i)+"_2018" for i in cols]
resp_2018_df = resp_2018_df.rename(columns = {'FIPS_2018':'FIPS'})

cols = resp_2017_df.columns
resp_2017_df.columns = [str(i)+"_2017" for i in cols]
resp_2017_df = resp_2017_df.rename(columns = {'FIPS_2017':'FIPS'})

# MERGE
resp_df = pd.merge(resp_2017_df, resp_2018_df, on=['FIPS'])

## Create new column
resp_df['NumUsers/NumBeneficiaries_2017'] = resp_df['Number_of_Users_2017']/resp_df['Number_of_Fee-for-Service_Beneficiaries_2017']
resp_df['NumUsers/NumBeneficiaries_2018'] = resp_df['Number_of_Users_2018']/resp_df['Number_of_Fee-for-Service_Beneficiaries_2018']
## Create new column
resp_df['Payment/NumBeneficiaries_2017'] = resp_df['Total_Payment_2017']/resp_df['Number_of_Fee-for-Service_Beneficiaries_2017']
resp_df['Payment/NumBeneficiaries_2018'] = resp_df['Total_Payment_2018']/resp_df['Number_of_Fee-for-Service_Beneficiaries_2018']

## Create dataframe
#1.
## DV: 'TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018 '
## IV: 'NumUsers/NumBeneficiaries_2017', 'NumUsers/NumBeneficiaries_2018'
resp_1_df = resp_df[['FIPS','TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018',
                    'NumUsers/NumBeneficiaries_2017', 'NumUsers/NumBeneficiaries_2018']]
#2.
## DV: 'TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018 '
## IV: 'Payment/NumBeneficiaries_2017', 'Payment/NumBeneficiaries_2018'
resp_2_df = resp_df[['FIPS','TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018',
                    'Payment/NumBeneficiaries_2017', 'Payment/NumBeneficiaries_2018']]

In [346]:
print(np.corrcoef(resp_1_df['NumUsers/NumBeneficiaries_2017'].values, resp_1_df['TOTAL_PAYMENTS_2017'].values)[0][-1])
print(np.corrcoef(resp_1_df['NumUsers/NumBeneficiaries_2018'].values, resp_1_df['TOTAL_PAYMENTS_2018'].values)[0][-1])
print(np.corrcoef(resp_2_df['Payment/NumBeneficiaries_2017'].values, resp_2_df['TOTAL_PAYMENTS_2017'].values)[0][-1])
print(np.corrcoef(resp_2_df['Payment/NumBeneficiaries_2018'].values, resp_2_df['TOTAL_PAYMENTS_2018'].values)[0][-1])

-0.28650956587052084
-0.2882811727670624
-0.24184918689684087
-0.18413714568544007


In [348]:
# rename columns for merge
cols = kidney_2018_df.columns
kidney_2018_df.columns = [str(i)+"_2018" for i in cols]
kidney_2018_df = kidney_2018_df.rename(columns = {'FIPS_2018':'FIPS'})

cols = kidney_2017_df.columns
kidney_2017_df.columns = [str(i)+"_2017" for i in cols]
kidney_2017_df = kidney_2017_df.rename(columns = {'FIPS_2017':'FIPS'})

# MERGE
kidney_df = pd.merge(kidney_2017_df, kidney_2018_df, on=['FIPS'])

## Create new column
kidney_df['NumUsers/NumBeneficiaries_2017'] = kidney_df['Number_of_Users_2017']/kidney_df['Number_of_Fee-for-Service_Beneficiaries_2017']
kidney_df['NumUsers/NumBeneficiaries_2018'] = kidney_df['Number_of_Users_2018']/kidney_df['Number_of_Fee-for-Service_Beneficiaries_2018']
## Create new column
kidney_df['Payment/NumBeneficiaries_2017'] = kidney_df['Total_Payment_2017']/kidney_df['Number_of_Fee-for-Service_Beneficiaries_2017']
kidney_df['Payment/NumBeneficiaries_2018'] = kidney_df['Total_Payment_2018']/kidney_df['Number_of_Fee-for-Service_Beneficiaries_2018']

## Create dataframe
#1.
## DV: 'TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018 '
## IV: 'NumUsers/NumBeneficiaries_2017', 'NumUsers/NumBeneficiaries_2018'
kidney_1_df = kidney_df[['FIPS','TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018',
                    'NumUsers/NumBeneficiaries_2017', 'NumUsers/NumBeneficiaries_2018']]
#2.
## DV: 'TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018 '
## IV: 'Payment/NumBeneficiaries_2017', 'Payment/NumBeneficiaries_2018'
kidney_2_df = kidney_df[['FIPS','TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018',
                    'Payment/NumBeneficiaries_2017', 'Payment/NumBeneficiaries_2018']]

In [350]:
print(np.corrcoef(kidney_1_df['NumUsers/NumBeneficiaries_2017'].values, kidney_1_df['TOTAL_PAYMENTS_2017'].values)[0][-1])
print(np.corrcoef(kidney_1_df['NumUsers/NumBeneficiaries_2018'].values, kidney_1_df['TOTAL_PAYMENTS_2018'].values)[0][-1])
print(np.corrcoef(kidney_2_df['Payment/NumBeneficiaries_2017'].values, kidney_2_df['TOTAL_PAYMENTS_2017'].values)[0][-1])
print(np.corrcoef(kidney_2_df['Payment/NumBeneficiaries_2018'].values, kidney_2_df['TOTAL_PAYMENTS_2018'].values)[0][-1])

-0.2498495239585422
-0.25443721612739
-0.21036438555261203
-0.16250994761756982


In [351]:
# rename columns for merge
cols = infect_2018_df.columns
infect_2018_df.columns = [str(i)+"_2018" for i in cols]
infect_2018_df = infect_2018_df.rename(columns = {'FIPS_2018':'FIPS'})

cols = infect_2017_df.columns
infect_2017_df.columns = [str(i)+"_2017" for i in cols]
infect_2017_df = infect_2017_df.rename(columns = {'FIPS_2017':'FIPS'})

# MERGE
infect_df = pd.merge(infect_2017_df, infect_2018_df, on=['FIPS'])

## Create new column
infect_df['NumUsers/NumBeneficiaries_2017'] = infect_df['Number_of_Users_2017']/infect_df['Number_of_Fee-for-Service_Beneficiaries_2017']
infect_df['NumUsers/NumBeneficiaries_2018'] = infect_df['Number_of_Users_2018']/infect_df['Number_of_Fee-for-Service_Beneficiaries_2018']
## Create new column
infect_df['Payment/NumBeneficiaries_2017'] = infect_df['Total_Payment_2017']/infect_df['Number_of_Fee-for-Service_Beneficiaries_2017']
infect_df['Payment/NumBeneficiaries_2018'] = infect_df['Total_Payment_2018']/infect_df['Number_of_Fee-for-Service_Beneficiaries_2018']

## Create dataframe
#1.
## DV: 'TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018 '
## IV: 'NumUsers/NumBeneficiaries_2017', 'NumUsers/NumBeneficiaries_2018'
infect_1_df = infect_df[['FIPS','TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018',
                    'NumUsers/NumBeneficiaries_2017', 'NumUsers/NumBeneficiaries_2018']]
#2.
## DV: 'TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018 '
## IV: 'Payment/NumBeneficiaries_2017', 'Payment/NumBeneficiaries_2018'
infect_2_df = infect_df[['FIPS','TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018',
                    'Payment/NumBeneficiaries_2017', 'Payment/NumBeneficiaries_2018']]

In [352]:
print(np.corrcoef(infect_1_df['NumUsers/NumBeneficiaries_2017'].values, infect_1_df['TOTAL_PAYMENTS_2017'].values)[0][-1])
print(np.corrcoef(infect_1_df['NumUsers/NumBeneficiaries_2018'].values, infect_1_df['TOTAL_PAYMENTS_2018'].values)[0][-1])
print(np.corrcoef(infect_2_df['Payment/NumBeneficiaries_2017'].values, infect_2_df['TOTAL_PAYMENTS_2017'].values)[0][-1])
print(np.corrcoef(infect_2_df['Payment/NumBeneficiaries_2018'].values, infect_2_df['TOTAL_PAYMENTS_2018'].values)[0][-1])

-0.215192794914998
-0.22256447854567393
-0.18245512232449135
-0.14307668037309998


In [353]:
# rename columns for merge
cols = circ_2018_df.columns
circ_2018_df.columns = [str(i)+"_2018" for i in cols]
circ_2018_df = circ_2018_df.rename(columns = {'FIPS_2018':'FIPS'})

cols = circ_2017_df.columns
circ_2017_df.columns = [str(i)+"_2017" for i in cols]
circ_2017_df = circ_2017_df.rename(columns = {'FIPS_2017':'FIPS'})

# MERGE
circ_df = pd.merge(circ_2017_df, circ_2018_df, on=['FIPS'])

## Create new column
circ_df['NumUsers/NumBeneficiaries_2017'] = circ_df['Number_of_Users_2017']/circ_df['Number_of_Fee-for-Service_Beneficiaries_2017']
circ_df['NumUsers/NumBeneficiaries_2018'] = circ_df['Number_of_Users_2018']/circ_df['Number_of_Fee-for-Service_Beneficiaries_2018']
## Create new column
circ_df['Payment/NumBeneficiaries_2017'] = circ_df['Total_Payment_2017']/circ_df['Number_of_Fee-for-Service_Beneficiaries_2017']
circ_df['Payment/NumBeneficiaries_2018'] = circ_df['Total_Payment_2018']/circ_df['Number_of_Fee-for-Service_Beneficiaries_2018']

## Create dataframe
#1.
## DV: 'TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018 '
## IV: 'NumUsers/NumBeneficiaries_2017', 'NumUsers/NumBeneficiaries_2018'
circ_1_df = circ_df[['FIPS','TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018',
                    'NumUsers/NumBeneficiaries_2017', 'NumUsers/NumBeneficiaries_2018']]
#2.
## DV: 'TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018 '
## IV: 'Payment/NumBeneficiaries_2017', 'Payment/NumBeneficiaries_2018'
circ_2_df = circ_df[['FIPS','TOTAL_PAYMENTS_2017', 'TOTAL_PAYMENTS_2018',
                    'Payment/NumBeneficiaries_2017', 'Payment/NumBeneficiaries_2018']]

In [354]:
print(np.corrcoef(circ_1_df['NumUsers/NumBeneficiaries_2017'].values, circ_1_df['TOTAL_PAYMENTS_2017'].values)[0][-1])
print(np.corrcoef(circ_1_df['NumUsers/NumBeneficiaries_2018'].values, circ_1_df['TOTAL_PAYMENTS_2018'].values)[0][-1])
print(np.corrcoef(circ_2_df['Payment/NumBeneficiaries_2017'].values, circ_2_df['TOTAL_PAYMENTS_2017'].values)[0][-1])
print(np.corrcoef(circ_2_df['Payment/NumBeneficiaries_2018'].values, circ_2_df['TOTAL_PAYMENTS_2018'].values)[0][-1])

-0.2597932786301336
-0.2618417613521999
-0.21961760117079707
-0.16821090819971618
